In [1]:
PROJECT_ID = "formidable-feat-466408-r6"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://agentic_ai_ebooks_bucket"  # @param {type:"string"}
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [2]:
from google import adk
from google.adk.agents import Agent
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part
from google.adk.sessions import VertexAiSessionService
from google.adk.tools import google_search

In [8]:
from vertexai import rag
from google.adk.tools.retrieval.vertex_ai_rag_retrieval import VertexAiRagRetrieval
from vertexai.generative_models import GenerativeModel, Tool
from google.genai import types # For types.Content
from vertexai import generative_models

In [10]:
from google.adk.tools.application_integration_tool.application_integration_toolset import ApplicationIntegrationToolset

In [11]:
PROJECT_ID = "formidable-feat-466408-r6"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://agentic_ai_ebooks_bucket"  # @param {type:"string"}

In [18]:
connector_tool = ApplicationIntegrationToolset(
    project=PROJECT_ID, # TODO: replace with GCP project of the connection
    location=LOCATION, #TODO: replace with location of the connection
    # connection=f"projects/{PROJECT_ID}/locations/{LOCATION}/connections/google-drive-connection", #TODO: replace with connection name "projects/genai-app-builder/locations/europe-central2/connections/gdrive-connection", ##
    connection="google-drive-connection",
    entity_operations={},##{"Entity_One": ["LIST","CREATE"], "Entity_Two": []},#empty list for actions means all operations on the entity are supported.
    actions=["GET_files"], #TODO: replace with actions
    ##service_account_credentials='{...}', # optional
    # tool_name="tool_list_gdrive_files",
    tool_instructions="Use this tool to list gdrive files"
)

In [19]:
agent_model = "gemini-2.0-flash-001"
AGENT_APP_NAME = "Gdrive_Agent"

In [24]:
tools_list = await connector_tool.get_tools()

In [27]:
gdrive_agent = Agent(
        model=agent_model,
        name=AGENT_APP_NAME,
        description="You are helpful assitant",
        instruction="If they ask you how you were created, tell them you were created with the Google Agent Framework.",
        generate_content_config=types.GenerateContentConfig(temperature=0.2),
        tools = tools_list,
)

In [28]:
app = AdkApp(
   agent=gdrive_agent,
    enable_tracing=True# Required.
   # session_service_builder=session_service_builder,  # Optional.
)

In [46]:
session = app.create_session(user_id='user')

In [52]:
session

Session(id='9dc8a748-3fb2-4335-ad99-a50b19f26bec', app_name='default-app-name', user_id='user', state={}, events=[], last_update_time=1753511760.2651718)

In [54]:
for event in app.stream_query(
    user_id="1",
    # session_id=SESSION_ID, # Optional. you can pass in the session_id when querying the agent
    message="list files on my gdrive",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-7b4a719b-c7e4-421b-8bc2-116711f5c5a4', 'args': {'connector_input_payload': {}, 'connection_name': 'projects/formidable-feat-466408-r6/locations/us-central1/connections/google-drive-connection', 'service_name': 'projects/gc816baa67c8b42f8-tp/locations/us-central1/namespaces/cloudrun/services/runtime-tls', 'host': 'https://gc816baa67c8b42f8-tp-us-central1.us-central1-runtime-connectors-google.com', 'entity': None, 'operation': 'EXECUTE_ACTION', 'action': 'GET_files'}, 'name': 'drive_files_list'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 10, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 10}], 'prompt_token_count': 549, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 549}], 'total_token_count': 559, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-fd503888-6d59-4b5e-a54d-8a6b4a6fbb1c', 'author': 'Gdrive_Agent', 

In [30]:
session_service = VertexAiSessionService(project=PROJECT_ID, location=LOCATION)

In [39]:
# _session_id="session1"
# session = await session_service.create_session(app_name=app, user_id='user')

In [32]:
# runner = Runner(app_name=app, agent=gdrive_agent, session_service=session_service)

In [42]:
# def get_response(runner, session, message):
#     content = types.Content(role='user', parts=[types.Part(text=message)])
#     events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
#     final_response = None
#     for _, event in enumerate(events):
#         print(event)
#         is_final_response = event.is_final_response()
#         if is_final_response:
#             final_response = event.content.parts[0].text
#     return final_response

In [45]:
# resp = get_response(runner, session, "list files on my gdrive")
# resp

Exception in thread Thread-9 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/google/adk/runners.py", line 158, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/opt/conda/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 649, in run_until_complete
    return future.result()
  File "/opt/conda/lib/python3.10/site-packages/google/adk/runners.py", line 146, in _invoke_run_async
    async for event in self.run_async(
  File "/opt/conda/lib/python3.10/site-packages/google/adk/ru

In [44]:
resp